# Data Preprocessing

### Überblick über Daten

- **ACDH-CH-Website** - https://www.oeaw.ac.at/acdh (Tags bei Projekten und Tools) - Englisch
- **ARCHE** - https://arche.acdh.oeaw.ac.at (hasAppliedMethod, hasSubject, hasCategory) - Englisch/Deutsch
- **DARIAH Campus** - https://campus.dariah.eu/ (Topics bei Resources) - Englisch
- **dha Website** - https://digital-humanities.at/de/dha/projects (Tags bei Projekten) - Englisch/Deutsch
- **dha Taxonomy** - https://vocabs.acdh.oeaw.ac.at/dha_taxonomy - Englisch
- **GAMS** - https://gams.uni-graz.at/ (Tags/Keywords für Filter) - Englisch und Deutsch (Tags in Übersetzung vorhanden)
- **HowTo des ACDH-CH** - https://howto.acdh.oeaw.ac.at (Tags bei Posts) - Englisch/Deutsch
- **SSHOC** - sshoc-keyword.ttl from https://github.com/SSHOC/vocabularies/tree/main/SSHOpenMarketplace/sshoc-keyword - English
- **TaDiRAH** - https://vocabs.dariah.eu/tadirah/en/ - Englisch
- **TaDiRAH-Products** (modifizierte Variante, in Graz in Arbeit) - Englisch
- **Zotero von KONDE** - https://www.zotero.org/groups/1332658/konde (manuelle und automatische Tags zu Sekundärliteratur) - Englisch/Deutsch/Französisch/Spanisch/Niederländisch etc.

In [26]:
import csv
import json
import nltk
from tabulate import tabulate
from collections import Counter, defaultdict
import string
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import re
from rdflib import Graph
from rdfpandas.graph import to_dataframe
import pandas as pd
import rdflib

### ACDH-CH Website

In [27]:
#Import der Tags der ACDH-CH-Website (acdh)
#zusätzliche Informationen: Count für Tools, Count für Projekte, Count insgesamt

acdh_orig = {}

with open("Tags_ACDH-CH-Website.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")
        tag = element[0]
        count_tools = element[1]
        count_projects = element[2]
        count = element[3]
        if tag == "\ufeffTag" or tag == "Summe":
            continue
        else:
            acdh_orig[tag] = {"count_tools": count_tools, "count_projects": count_projects, "count": count}

acdh = {key.lower():value for (key,value) in acdh_orig.items()}

with open("acdhch_website.json", "w", encoding = "utf-8") as datei:
    json.dump(acdh, datei, ensure_ascii = False)

### ARCHE (hasAppliedMethod, hasSubject, hasCategory)

In [28]:
#Import der ARCHE-Tags (hasSubject + hasAppliedMethod + hasCategory)
#zusätzliche Informationen: Count, Property
#Behandlung der drei Properties als drei verschiedene Vokabularien

arche_hasSubject_orig = {}
arche_hasAppliedMethod_orig = {}
arche_hasCategory_orig = {}
counter = 0

with open("ARCHE_erweitert_2022-09-29.csv", encoding = "utf-8") as datei:
    for row in datei:
        counter = counter + 1
        element = row.replace("Ã¤", "ä").replace("Ã¶", "ö").replace("Ã¼", "ü").replace("ÃŸ", "ß").replace("Ã–", "Ö").replace("Ã„", "Ä").replace("Ãœ", "Ü").replace("Ã¡", "á").replace("ÄŸ", "ğ").replace("Ã©", "é").strip().split(";")
        if element[0] == "\ufeffsubject":
            continue
        else:
            tag = element[0]
            count = element[1]
            if element[2] == "hasSubject":
                arche_hasSubject_orig[tag] = count
            elif element[2] == "hasAppliedMethod":
                arche_hasAppliedMethod_orig[tag] = count
            elif element[2] == "hasCategory":
                arche_hasCategory_orig[tag] = count
            else:
                print("Fehler - unbekannte Kategorie: " + element[2] + " (Zeile: " + str(counter) + ")")

#manuelle Aufnahme von einem Fall mit ";" innerhalb von subject (s. Fehlermeldung im Output)
arche_hasSubject_orig["25er-Ausschuss; Fünfundzwanziger-Ausschuss"] = 1

arche_hasAppliedMethod = {key.lower():value for (key,value) in arche_hasAppliedMethod_orig.items()}
arche_hasCategory = {key.lower():value for (key,value) in arche_hasCategory_orig.items()}
arche_hasSubject = {key.lower():value for (key,value) in arche_hasSubject_orig.items()}

#manuelle Nachkorrektur des Counts bei Dopplungen (Hintergrund: erster Key wurde hier automatisch durch zweiten Key überschrieben)

arche_hasSubject["military frontier"] = 9
arche_hasSubject["bibliography"] = 9

arche = {}
for key in arche_hasAppliedMethod:
    if key not in arche:
        arche[key] = {}
for key in arche_hasCategory:
    if key not in arche:
        arche[key] = {}
for key in arche_hasSubject:
    if key not in arche:
        arche[key] = {}

with open("arche.json", "w", encoding = "utf-8") as datei:
    json.dump(arche, datei, ensure_ascii = False)

with open("arche_hasAppliedMethod.json", "w", encoding = "utf-8") as datei:
    json.dump(arche_hasAppliedMethod, datei, ensure_ascii = False)

with open("arche_hasCategory.json", "w", encoding = "utf-8") as datei:
    json.dump(arche_hasCategory, datei, ensure_ascii = False)

with open("arche_hasSubject.json", "w", encoding = "utf-8") as datei:
    json.dump(arche_hasSubject, datei, ensure_ascii = False)

Fehler - unbekannte Kategorie: 1 (Zeile: 572)


### DARIAH Campus

In [29]:
#Import von DARIAH-Campus (dc)
#zusätzliche Informationen: Count

with open("DARIAH-Campus_Topics.json", encoding = "utf-8") as datei:
    dc_orig = json.load(datei)

dc = {key.lower():value for (key,value) in dc_orig.items()}

with open("dariah_campus.json", "w", encoding = "utf-8") as datei:
    json.dump(dc, datei, ensure_ascii = False)

### dha Website

In [30]:
#Import der dha-Website-Tags
#Notiz: eine Dopplung ("political science" zweimal, mit je Count = 1) direkt im .csv korrigiert

dha_orig = {}

with open("dha-tags-count_20220928.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(",")
        if element[0] == "Name":
            continue
        else: 
            tag = element[0]
            count = element[1]
            dha_orig[tag] = count

#manuelle Korrektur eines Eintrags mit Beistrichen im Tag
del dha_orig["Formats"]
dha_orig["Formats, Protocols, Standards"] = 1

dha = {key.lower():value for (key,value) in dha_orig.items()}

with open("dha_website.json", "w", encoding = "utf-8") as datei:
    json.dump(dha, datei, ensure_ascii = False)

### dha Taxonomy

In [31]:
g = rdflib.Graph()
g.parse('dhataxonomy.ttl', format = 'ttl')
df = to_dataframe(g)
df.to_csv('dha_taxonomy.csv', sep="#")

keywords = {}

with open("dha_taxonomy.csv", encoding = "utf-8") as datei:
    for row in datei:
        liste = row.split("#")
        tag = liste[9].strip()
        link = liste[0].strip()
        creator = liste[1].strip()
        definition = liste[6].strip()
        tag_cleaned = tag.replace("_", " ").replace("-", " ").lower()
        if tag != r"skos:prefLabel{Literal}@en" and tag != "":
            keywords[tag_cleaned] = {"link": link, "creator": creator, "definition": definition}

with open("dha_taxonomy.json", "w", encoding = "utf-8") as datei:
    json.dump(keywords, datei, ensure_ascii = False)

### GAMS

In [32]:
#Import der Tags von GAMS
#zusätzliche Informationen: Sprache (de/en), Kategorie (Disziplin/Thema/Quellenmaterial/Methode/Periode), ID (selbst hinzugefügt, um Übersetzungen wiederzufinden)

gams_orig = {}

with open("GAMS.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")
        tag = element[0]
        lang = element[1] #Sprache
        cat = element[2] #Kategorie
        id = element[3] #ID
        if tag == "\ufeffTag":
            continue
        else:
            gams_orig[tag] = {"lang": lang, "cat": cat, "id": id}

gams = {key.lower():value for (key,value) in gams_orig.items()}

#manuelle Nachkorrektur des Counts bei Dopplungen (Hintergrund: erster Key wurde hier automatisch durch zweiten Key überschrieben)
gams["numismatics"]["cat"] = ["Disziplin", "Thema"]
gams["numismatics"]["id"] = ["16", "30"]

with open("gams.json", "w", encoding = "utf-8") as datei:
    json.dump(gams, datei, ensure_ascii = False)

### HowTo (ACDH-CH)

In [33]:
#Import der manuell ergänzten Version der Tags des ACDH-CH HowTo (ht)
#zusätzliche Informationen: Description, Count

with open("HowTo_ergänzt.json", encoding = "utf-8") as datei:
    ht_orig = json.load(datei)

ht = {key.lower():value for (key,value) in ht_orig.items()}

with open("howto.json", "w", encoding = "utf-8") as datei:
    json.dump(ht, datei, ensure_ascii = False)

### SSHOC

In [41]:
g = rdflib.Graph()
g.parse('sshoc-keyword.ttl', format = 'ttl')
df = to_dataframe(g)
df.to_csv('sshoc-keywords.csv', sep="#")

keywords = {}

with open("sshoc-keywords.csv", encoding = "utf-8") as datei:
    for row in datei:
        tag = row.split("#")[6]
        tag_cleaned = tag.replace("_", " ").replace("-", " ").replace("SSK:", "").lower().strip()
        if tag != r"rdfs:label{Literal}@en":
            keywords[tag_cleaned] = {}

with open("SSHOC.json", "w", encoding = "utf-8") as datei:
    json.dump(keywords, datei, ensure_ascii = False)

### TaDiRAH Processes

In [35]:
#Import der TaDiRAH-Tags
#zusätzliche Information: Synonym, falls es sich bei Tag um alternative Bezeichnung handelt (z.B. OCR --> Optical Character Recognition)

tadirah_orig = {}

with open("TaDiRAH.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")
        tag = element[0]
        synonym = element[1] 
        if tag == "\ufeffTag":
            continue
        else:
            tadirah_orig[tag] = {"synonym": synonym}

tadirah = {key.lower():value for (key,value) in tadirah_orig.items()}

with open("tadirah_processes.json", "w", encoding = "utf-8") as datei:
    json.dump(tadirah, datei, ensure_ascii = False)

### TaDiRAH Products

In [36]:
#Import der TaDiRAH-Product-Tags
#zusätzliche Information: Level in Tadirah, Synonym, gemappter Terminus in dha, gemappter Terminus in Wikidata

tadirah_prod_orig = {}

with open("TaDiRAH_products.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip().split(";")
        tag = element[1].strip("()?") #aktuell noch Klammern und Fragezeichen im GoogleSheet - hier herausgelöscht
        level = element[2] #Level in Tadirah (wenn ein Tag auf mehreren Leveln vorkommt, werden diese in einer Liste gesammelt)
        synonym = element[3] #etwaige eingetragene Synonyme - z.B. Addition zu Adding
        dha_match = element[4] #gemappter Terminus im dha-Tagset
        #hier noch wiki-data-match einfügen (sobald in csv übertragen)
        if tag == "Tag" or tag == "":
            continue
        else:
            if tag not in tadirah_prod_orig.keys():
                tadirah_prod_orig[tag] = {"level": [level], "synonym": synonym, "dha_match": dha_match}
            else:
                tadirah_prod_orig[tag]["level"].append(level)

tadirah_prod = {key.lower():value for (key,value) in tadirah_prod_orig.items()}

with open("tadirah_products.json", "w", encoding = "utf-8") as datei:
    json.dump(tadirah_prod, datei, ensure_ascii = False)

### Zotero (KONDE)

In [37]:
#Import der Tags von KONDE-Zotero-Gruppe
#Zusatzinformation: Sprache (manuell jweils als en, de, en/de oder other (d.h. andere Sprache) klassifiziert)

zotero_orig = {}

##Extraschritt bei Zotero: Cleaning von metatextuellen Markierungen folgender Art sowie Trennung von mehreren Tags in einem Tag
metamarkierungen = ["act_", "activity: ", "meta_", "university: ", "dm_", "type: ", "t_", "obj_", "object: ", "goal_", "field: ", "supervisor: "]
seperatoren = [" / ", ", ", " - ", " & ", "&"]

with open("Zotero.csv", encoding = "utf-8") as datei:
    for row in datei:
        element = row.strip("\ufeff").strip().split(";")
        tag = element[0]
        lang = element[1]
        if tag == "Tag" and lang == "Sprache":
            continue
        for markierung in metamarkierungen:
            tag = tag.replace(markierung, "")
        if re.findall("[a-z][A-Z]", tag) and tag != "Thesis (PhD level)":
            for match in re.finditer(("[a-z][A-Z]"), tag):
                start = match.start() #gibt Index des ersten Zeichens von Match aus
                end = match.end()
                tag = tag[:start+1] + " " + tag[end-1:]
        #for sep in seperatoren:
            #if sep in tag:
                #tags = tag.split(sep)
            
        zotero_orig[tag] = {"lang": lang} #in Zukunft noch weitere Informationen hinzufügbar

zotero = {key.lower():value for (key,value) in zotero_orig.items()}

with open("zotero_konde.json", "w", encoding = "utf-8") as datei:
    json.dump(zotero, datei, ensure_ascii = False)